In [ ]:
# Analyze ImageNet expert experiment

In [ ]:
import pandas as pd  
import glob
import os

tasks = ['Natural', 'Dog', 'Adversarial_Nat', 'Adversarial_Dog']
dataset_path = '/home/dexter/Downloads/Human_experiments/Dataset'
ground_truth = dict()

# Build the ground truth dictionary 

CORRECT_BIN1_IMAGES = 'correct_bin1_images/'
CORRECT_BIN2_IMAGES = 'correct_bin2_images/'
CORRECT_BIN3_IMAGES = 'correct_bin3_images/'
WRONG_BIN1_IMAGES = 'wrong_bin1_images/'
WRONG_BIN2_IMAGES = 'wrong_bin2_images/'
WRONG_BIN3_IMAGES = 'wrong_bin3_images/'
bin_images = [CORRECT_BIN1_IMAGES, CORRECT_BIN2_IMAGES, CORRECT_BIN3_IMAGES, WRONG_BIN1_IMAGES, WRONG_BIN2_IMAGES, WRONG_BIN3_IMAGES]
easy_bins = [WRONG_BIN1_IMAGES, CORRECT_BIN3_IMAGES]
hard_bins = [CORRECT_BIN1_IMAGES, WRONG_BIN3_IMAGES]
norm_bins = [CORRECT_BIN2_IMAGES, WRONG_BIN2_IMAGES]


for task in tasks:
    ground_truth[task] = dict()
    task_dataset_path = os.path.join(dataset_path,task)
    for CORRECT_BIN_IMAGES in [CORRECT_BIN1_IMAGES, CORRECT_BIN2_IMAGES, CORRECT_BIN3_IMAGES]:
        bin_gt_dict = dict()
        correct_bin_images_path = os.path.join(task_dataset_path, CORRECT_BIN_IMAGES)
        correct_bin_images = [path.split(CORRECT_BIN_IMAGES)[1] for path in glob.glob(correct_bin_images_path + '/*.*')]
        for correct_bin_image in correct_bin_images:
            bin_gt_dict[correct_bin_image] = 'Yes'
        
        ground_truth[task][CORRECT_BIN_IMAGES] = bin_gt_dict
        
    for WRONG_BIN_IMAGES in [WRONG_BIN1_IMAGES, WRONG_BIN2_IMAGES, WRONG_BIN3_IMAGES]:
        bin_gt_dict = dict()
        wrong_bin_images_path = os.path.join(task_dataset_path, WRONG_BIN_IMAGES)
        wrong_bin_images = [path.split(WRONG_BIN_IMAGES)[1] for path in glob.glob(wrong_bin_images_path + '/*.*')]
        for wrong_bin_image in wrong_bin_images:
            bin_gt_dict[wrong_bin_image] = 'No'
        
        ground_truth[task][WRONG_BIN_IMAGES] = bin_gt_dict

In [ ]:
import pandas as pd  
import operator
import os
import math
import statistics
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
%matplotlib inline

# Check if a number is float?
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
# Task aliases assigned by Gorilla
exp_hierarchy = ['randomiser-tjl7', {'GradCAM': 'counterbalance-ao9d', 'NNs': 'counterbalance-ao9d'}]

methods = ['GradCAM', 'NNs']
dataset_path = '/home/dexter/Downloads/Human_experiments/Dataset'

# Loop all csv file, each csv file is an explanation method
files = []

exp = 'Natural'
for file in glob.glob('Expert_Data/{}/*.*'.format(exp)):
    if '.csv' in file:
        files.append(file)

if exp == 'Natural':
    tasks = ['Natural', 'Adversarial_Nat']
    threshold = 8
else:
    tasks = ['Dog', 'Adversarial_Dog']
    threshold = 8
    
csv_file = open('csv_files/{}_bad_users.csv'.format(tasks[0]), 'w')
final_result = dict()
trial_cnt = dict()

# Initialize the dictionary for users' responses (answer)
for task in tasks:
    final_result[task] = dict()
    trial_cnt[task] = dict()
    for method in methods:
        final_result[task][method] = dict()
        trial_cnt[task][method] = dict()
        for bin_image in bin_images:
            final_result[task][method][bin_image] = 0
            trial_cnt[task][method][bin_image] = 0

# Correct answer dictionary in validation 
val_correct_dict = dict()
# Incorrect answer dictionary in validation 
val_incorrect_trials_dict = dict()

test_correct_trials_dict = dict()
test_incorrect_trials_dict = dict()

# Numbers of users for methods
user_cnt_dict = dict()

# Reaction time dictionary 
users_avg_reaction_time_dict = dict()
users_stdev_reaction_time_dict = dict()
below_stdev_reaction_time_dict = dict()

# Counter-balances for methods
counter_balances_dict = dict()

# Numbers of good users for methods
good_user_cnt_dict = dict()
good_user_cnt = 0
bad_user_cnt = 0
users_reaction_times = []
user_dict = dict()

for file in files:
    reaction_time_correct_cnt_dict = dict()
    users_reaction_time = []
    users_val_incorrect_trials = []
    counter_balances = []
    user_cnt = 0
    df = pd.read_csv(file)
#     reaction_time = (df['Reaction Time'].sum()/1000)/60 # in minutes
    for index, row in df.iterrows():
        
        # Start the spreadsheet for a user
        if row['Event Index'] != 'END OF FILE' and int(row['Event Index']) == 1:
            user_cnt += 1
#             print('-------- STATISTICS FOR USER NUMBER {} --------'.format(user_cnt))
            reaction_time = 0
            reaction_times = []

            val_correct = 0
            val_incorrect = 0
            val_incorrect_trials = []
            val_trial_cnt = 0
            
            test_reaction_time = 0
            val_reaction_time = 0

            test_trial_answers = {}

            task_name = row['Task Name']
#             print(task_name)
            method = task_name.split('_')[1]
            if method == 'Confidence':
                method = 'Conf'

            task = task_name.split('_')[0]
            counter_balance = int(row[exp_hierarchy[1][method]])
# #             print(task_name, method, counter_balance)
#             counter_balances.append(counter_balance)

            public_id = row['Participant Public ID']
    
            if task not in user_dict:
                user_dict[task] = dict()
                    
            if method not in user_dict[task]:
                user_dict[task][method] = dict()
            if public_id not in user_dict[task][method]:
                user_dict[task][method][public_id] = dict()
                user_dict[task][method][public_id]['Trials'] = dict()
                user_dict[task][method][public_id]['Prior Knowledge'] = dict()
                user_dict[task][method][public_id]['Counter balance'] = counter_balance
                user_dict[task][method][public_id]['Known'] = 0
                user_dict[task][method][public_id]['Unknown'] = 0
        
        trial_time = row['Reaction Time']
        
#         # Get the reaction time for each screen
#         if (isinstance(trial_time, str) and is_float(trial_time)) or (isinstance(trial_time, float) and not math.isnan(trial_time)):
#             reaction_time += float(trial_time)
#             reaction_times.append(float(trial_time))
            
        # Check users' responses in validation
        if row['display'] == 'Validation' and row['Screen Name'] == 'Screen 3':
            val_trial_cnt += 1
            if row['Correct'] == 1:
                val_correct += 1
            elif row['Incorrect'] == 1:
                val_incorrect += 1
                val_incorrect_trials.append(val_trial_cnt)
            else:
                raise ValueError("Wrong value!")
                
            user_dict[task][method][public_id]['Validation Correct'] = val_correct
            user_dict[task][method][public_id]['Validation Incorrect'] = val_incorrect
            user_dict[task][method][public_id]['Incorrect Validation Trials'] = val_incorrect_trials
            
        elif row['display'] == 'Trial' and row['Screen Name'] == 'Screen 1':
            prior_knowledge = row['Response']
                
        # Check users' responses in test
        elif row['display'] == 'Trial' and row['Screen Name'] == 'Screen 3':
            file_name = row['file_name' + str(counter_balance)]
            file_name = (file_name.split('.jpeg')[0]).split('_')[:-1]
            file_name = '_'.join(file_name) + '.jpeg'
            test_trial_answers[file_name] = row['Response']
            
            user_dict[task][method][public_id]['Trials'][file_name] = row['Response']
            user_dict[task][method][public_id]['Prior Knowledge'][file_name] = prior_knowledge
        
        # Time from instructions -> the end of Validation
        if row['display'] != 'Trial':
            if (isinstance(row['Reaction Time'], str) and is_float(row['Reaction Time'])) or (isinstance(row['Reaction Time'], float) and not math.isnan(row['Reaction Time'])):
                val_reaction_time += float(row['Reaction Time'])
        
        # End the spreadsheet for a user
        if row['Trial Number'] == 'END TASK':
            reaction_time = float(row['Reaction Time'])
            
            # Validation and Test are combined
            user_dict[task][method][public_id]['Validation Reaction Time'] = val_reaction_time
            user_dict[task][method][public_id]['Test Reaction Time'] = reaction_time - test_reaction_time
            


User_dict
- Natural
    - Methods
        - User_IDs
            - ['Trials', '
            - Counter balance', 
            - 'Validation Correct', ' # The number of correct validation trials
            - Validation Incorrect', ' # The number of incorrect validation trials
            - Incorrect Validation Trials', ' # The IDs of incorrect validation trials
            - Validation Reaction Time', '#  # Time from instructions -> the end of Validation
            - Test Reaction Time', ' # Time taken for 30 test trials
            - Quality']) # This user qualified the validation or not.

In [ ]:
# print(user_dict['Natural']['SOD'].keys())
# print(len(user_dict['Natural']['Conf']))
test_correct_trials_dict = dict()

# Initialize the dictionary for users' responses (answer)
for task in tasks:
    final_result[task] = dict()
    trial_cnt[task] = dict()
    for method in methods:
        final_result[task][method] = dict()
        trial_cnt[task][method] = dict()
        for bin_image in bin_images:
            final_result[task][method][bin_image] = 0
            trial_cnt[task][method][bin_image] = 0

task = exp
for method in methods:
    print(method)
    good_user_cnt = 0
    hacking_user_cnt = 0
    correct_cnt_list = []
    
    if method not in test_correct_trials_dict:
        test_correct_trials_dict[method] = dict()
    for user_id in user_dict[task][method].keys():
#         print(user_id)
#         print(user_dict[task][method][user_id]['Known'])
        
        if user_dict[task][method][user_id]['Validation Correct'] >= threshold:
            good_user_cnt += 1
            user_dict[task][method][user_id]['Quality'] = 'Good'
        else:
            user_dict[task][method][user_id]['Quality'] = 'Bad'
#             if user_dict[task][method][user_id]['Test Reaction Time'] < 1*user_dict[task][method][user_id]['Validation Reaction Time']:
#                 hacking_user_cnt += 1
            continue
        
        trials = user_dict[task][method][user_id]['Trials']
        
        # Initialize: The correct times a trial gets {trial_name: times}
        for trial_key in trials.keys():
            if trial_key not in test_correct_trials_dict[method]:
                test_correct_trials_dict[method][trial_key] = dict()
                test_correct_trials_dict[method][trial_key]['Count'] = 0
                test_correct_trials_dict[method][trial_key]['Counter balance'] = list()
            test_correct_trials_dict[method][trial_key]['Counter balance'].append(user_dict[task][method][user_id]['Counter balance'])
                
        if task == 'Natural':
            task_list = ['Natural', 'Adversarial_Nat']
        elif task == 'Dog':
            task_list = ['Dog', 'Adversarial_Dog']
            
        # Record the number of correct answers of a user for every bin
        for sub_task in task_list: 
            user_dict[task][method][user_id][sub_task] = dict()
            exp_gt = ground_truth[sub_task]

            for CORRECT_BIN_IMAGES in [CORRECT_BIN1_IMAGES, CORRECT_BIN2_IMAGES, CORRECT_BIN3_IMAGES]:
                shared_items = {k: exp_gt[CORRECT_BIN_IMAGES][k] for k in exp_gt[CORRECT_BIN_IMAGES] if k in trials and exp_gt[CORRECT_BIN_IMAGES][k] == trials[k]}
                user_dict[task][method][user_id][sub_task][CORRECT_BIN_IMAGES] = len(shared_items)
                
#                 print(len(shared_items))
                # Get: The correct times a trial gets {trial_name: times}
                for key in shared_items.keys():
                    test_correct_trials_dict[method][key]['Count'] += 1
                    if user_dict[task][method][user_id]['Prior Knowledge'][key] == 'Yes':
                        user_dict[task][method][user_id]['Known'] += 1
                    else:
                        user_dict[task][method][user_id]['Unknown'] += 1
                
                # Get: The prediction of AI on this image
                for key in exp_gt[CORRECT_BIN_IMAGES].keys():
                    if key in test_correct_trials_dict[method]:
                        test_correct_trials_dict[method][key]['Prediction'] = 'Correct'
                
                trial_cnt[sub_task][method][CORRECT_BIN_IMAGES] += len(set.intersection(set(ground_truth[sub_task][CORRECT_BIN_IMAGES].keys()), set(trials.keys())))
                
            for WRONG_BIN_IMAGES in [WRONG_BIN1_IMAGES, WRONG_BIN2_IMAGES, WRONG_BIN3_IMAGES]:
                shared_items = {k: exp_gt[WRONG_BIN_IMAGES][k] for k in exp_gt[WRONG_BIN_IMAGES] if k in trials and exp_gt[WRONG_BIN_IMAGES][k] == trials[k]}
                user_dict[task][method][user_id][sub_task][WRONG_BIN_IMAGES] = len(shared_items)
#                 print(len(shared_items))
                # Get: The correct times a trial gets {trial_name: times}                    
                for key in shared_items.keys():
                    test_correct_trials_dict[method][key]['Count'] += 1
                    if user_dict[task][method][user_id]['Prior Knowledge'][key] == 'Yes':
                        user_dict[task][method][user_id]['Known'] += 1
                    else:
                        user_dict[task][method][user_id]['Unknown'] += 1
                    
                # Get: The prediction of AI on this image
                for key in exp_gt[WRONG_BIN_IMAGES].keys():
                    if key in test_correct_trials_dict[method]:
                        test_correct_trials_dict[method][key]['Prediction'] = 'Wrong'

                trial_cnt[sub_task][method][WRONG_BIN_IMAGES] += len(set.intersection(set(ground_truth[sub_task][WRONG_BIN_IMAGES].keys()), set(trials.keys())))
#             print(user_dict[task][method][user_id]['Known'])

        print('Method {}: user_id: {} Validation scores: {}'.format(method, user_id, user_dict[task][method][user_id]['Validation Correct']))
        print('Correct count = {}'.format(sum(user_dict[exp][method][user_id][tasks[0]].values()) + sum(user_dict[exp][method][user_id][tasks[1]].values())))
        correct_cnt = sum(user_dict[exp][method][user_id][tasks[0]].values()) + sum(user_dict[exp][method][user_id][tasks[1]].values())
        correct_cnt_list.append((correct_cnt/30)*100)
        
    print('{}/{} good users of {}'.format(good_user_cnt, len(user_dict[task][method]), method))
    print('std_dev = {}'.format(statistics.stdev(correct_cnt_list)))
    print('mean = {}'.format(statistics.mean(correct_cnt_list)))
#     print('{}/{} hacking users of {}'.format(hacking_user_cnt, len(user_dict[task][method]), method))

In [ ]:
# Q1
acc_dict = dict()
print('Q1: Are heatmaps more effective than 3-NNs in improving AI+human accuracy in classifying random ImageNet images?')
for task in tasks:
    acc_dict[task] = dict()
    print("In {} images:".format(task.split('_')[0]))
    accuracies = []
    best_acc = 0
    best_method = ''
    for method in methods:
        correct_cnt = 0
        known_crt_cnt = 0
        unknown_crt_cnt = 0
        for user_id in user_dict[exp][method].keys():
            if user_dict[exp][method][user_id]['Quality'] == 'Good':
                correct_cnt += sum(user_dict[exp][method][user_id][task].values())
                known_crt_cnt += user_dict[exp][method][user_id]['Known']
                unknown_crt_cnt += user_dict[exp][method][user_id]['Unknown']

#                 print(sum(user_dict[exp][method][user_id][task].values()), user_dict[exp][method][user_id]['Known'], user_dict[exp][method][user_id]['Unknown'])

        total_cnt = sum(trial_cnt[task][method].values())
#         print("Correct count: {} on Total count: {}".format(correct_cnt, total_cnt))
        print(correct_cnt, total_cnt)
        if total_cnt:
            acc = correct_cnt*100/total_cnt
            acc_dict[task][method] = round(acc,2) # limit to 2 decimals
            accuracies.append(acc)
            print("Task: {} | Method: {} | Accuracy: {:.2f}%".format(task, method, correct_cnt*100/total_cnt))
            if acc > best_acc:
                best_acc = acc
                best_method = method
                
    print("Answer: The best method is {} with an accuracy of {:.2f}%".format(best_method, best_acc))
    print("Answer: Mean accuracy is {:.2f}%".format(sum(accuracies)/len(accuracies)))
    
print(acc_dict)

known_crt_acc = []
unknown_crt_acc = []
for method in methods:
    correct_cnt = 0
    known_crt_cnt = 0
    unknown_crt_cnt = 0
    total_known_cnt = 0
    total_unknown_cnt = 0
    for user_id in user_dict[exp][method].keys():
        if user_dict[exp][method][user_id]['Quality'] == 'Good':
            known_crt_cnt += user_dict[exp][method][user_id]['Known']
            unknown_crt_cnt += user_dict[exp][method][user_id]['Unknown']
            total_known_cnt += sum(value == 'Yes' for value in user_dict[exp][method][user_id]['Prior Knowledge'].values())
            total_unknown_cnt += sum(value == 'No' for value in user_dict[exp][method][user_id]['Prior Knowledge'].values())
    print(known_crt_cnt, total_known_cnt, unknown_crt_cnt, total_unknown_cnt)
    known_crt_acc.append(known_crt_cnt*100/total_known_cnt)
    unknown_crt_acc.append(unknown_crt_cnt*100/total_unknown_cnt)
    print("On Known images: Method: {} | Accuracy: {:.2f}%".format(method, known_crt_cnt*100/total_known_cnt))
    print("On Unknown images: Method: {} | Accuracy: {:.2f}%".format(method, unknown_crt_cnt*100/total_unknown_cnt))
                           
print("Answer: Known Mean accuracy is {:.2f}%".format(sum(known_crt_acc)/len(known_crt_acc)))
print("Answer: Unnown Mean accuracy is {:.2f}%".format(sum(unknown_crt_acc)/len(unknown_crt_acc)))

In [ ]:
# Q2
print('Q2: Are heatmaps more effective than 3-NNs in improving AI+human accuracy in classifying natural ImageNet images that are harder or easier to AIs?')
print('-------------- RESULT OF EASY IMAGES --------------')
for task in tasks:
    print("In {} images:".format(task.split('_')[0]))
    accuracies = []
    best_acc = 0
    best_method = ''
    for method in methods:
        correct_cnt = 0
        total_cnt = 0
        for user_id in user_dict[exp][method].keys():
            if user_dict[exp][method][user_id]['Quality'] == 'Good':
                for easy_bin in [CORRECT_BIN3_IMAGES]:
                    correct_cnt += user_dict[exp][method][user_id][task][easy_bin]
        for easy_bin in easy_bins:            
#             total_cnt += trial_cnt[task][method][easy_bin]
            total_cnt = trial_cnt[task][method][CORRECT_BIN3_IMAGES]
        print(correct_cnt, total_cnt)
                    
#         print("Correct count: {} on Total count: {}".format(correct_cnt, total_cnt))
        if total_cnt:
            acc = correct_cnt*100/total_cnt
            accuracies.append(acc)
            print("Task: {} | Method: {} | Accuracy: {:.2f}%".format(task, method, acc))
            if acc > best_acc:
                best_acc = acc
                best_method = method
#     print("Answer: The best method is {} with an accuracy of {:.2f}%".format(best_method, best_acc))
#     print("Answer: Mean accuracy is {:.2f}%".format(sum(accuracies)/len(accuracies)))
    
print('-------------- RESULT OF HARD IMAGES --------------')
for task in tasks:
    print("In {} images:".format(task.split('_')[0]))
    accuracies = []
    best_acc = 0
    best_method = ''
    for method in methods:
        correct_cnt = 0
        total_cnt = 0
        for user_id in user_dict[exp][method].keys():
            if user_dict[exp][method][user_id]['Quality'] == 'Good':
                for hard_bin in [CORRECT_BIN1_IMAGES]:
                    correct_cnt += user_dict[exp][method][user_id][task][hard_bin]
        for hard_bin in hard_bins:            
#             total_cnt += trial_cnt[task][method][hard_bin]
            total_cnt = trial_cnt[task][method][CORRECT_BIN1_IMAGES]
                    
#         print("Correct count: {} on Total count: {}".format(correct_cnt, total_cnt))
        print(correct_cnt, total_cnt)
        if total_cnt:
            acc = correct_cnt*100/total_cnt
            accuracies.append(acc)
            print("Task: {} | Method: {} | Accuracy: {:.2f}%".format(task, method, acc))
            if acc > best_acc:
                best_acc = acc
                best_method = method
#     print("Answer: The best method is {} with an accuracy of {:.2f}%".format(best_method, best_acc))
#     print("Answer: Mean accuracy is {:.2f}%".format(sum(accuracies)/len(accuracies)))

In [ ]:
# Q3
print('Q3: Are heatmaps more effective than 3-NNs in improving AI+human accuracy when AIs are not sure i.e. 50/50?')
print('-------------- RESULT OF NORMAL IMAGES --------------')
for task in tasks:
    print("In {} images:".format(task.split('_')[0]))
    accuracies = []
    best_acc = 0
    best_method = ''
    for method in methods:
        correct_cnt = 0
        total_cnt = 0
        for user_id in user_dict[exp][method].keys():
            if user_dict[exp][method][user_id]['Quality'] == 'Good':
                for norm_bin in [CORRECT_BIN2_IMAGES]:
                    correct_cnt += user_dict[exp][method][user_id][task][norm_bin]
        for norm_bin in norm_bins:            
#             total_cnt += trial_cnt[task][method][norm_bin]
            total_cnt = trial_cnt[task][method][CORRECT_BIN2_IMAGES]
                    
#         print("Correct count: {} on Total count: {}".format(correct_cnt, total_cnt))
        print(correct_cnt, total_cnt)
        if total_cnt:
            acc = correct_cnt*100/total_cnt
            accuracies.append(acc)
            print("Task: {} | Method: {} | Accuracy: {:.2f}%".format(task, method, acc))
            if  acc > best_acc:
                best_acc = acc
                best_method = method
#     print("Answer: The best method is {} with an accuracy of {:.2f}%".format(best_method, best_acc))
#     print("Answer: Mean accuracy is {:.2f}%".format(sum(accuracies)/len(accuracies)))

In [ ]:
# Q4
print('Q4: Which methods help humans the most when AI is correct or wrong?')
print('-------------- RESULT OF CORRECT IMAGES --------------')
for task in tasks:
    print("In {} images:".format(task.split('_')[0]))
    best_acc = 0
    best_method = ''
    for method in methods:
        correct_cnt = 0
        total_cnt = 0
        for user_id in user_dict[exp][method].keys():
            if user_dict[exp][method][user_id]['Quality'] == 'Good':
                for CORRECT_BIN_IMAGES in [CORRECT_BIN1_IMAGES, CORRECT_BIN2_IMAGES, CORRECT_BIN3_IMAGES]:
                    correct_cnt += user_dict[exp][method][user_id][task][CORRECT_BIN_IMAGES]
        for CORRECT_BIN_IMAGES in [CORRECT_BIN1_IMAGES, CORRECT_BIN2_IMAGES, CORRECT_BIN3_IMAGES]:     
            total_cnt += trial_cnt[task][method][CORRECT_BIN_IMAGES]
                    
#         print("Correct count: {} on Total count: {}".format(correct_cnt, total_cnt))
        if total_cnt:
            print("Task: {} | Method: {} | Accuracy: {:.2f}%".format(task, method, correct_cnt*100/total_cnt))
            if correct_cnt*100/total_cnt > best_acc:
                best_acc = correct_cnt*100/total_cnt
                best_method = method
    
    print("Answer: The best method is {} with an accuracy of {:.2f}%".format(best_method, best_acc))
    
print('-------------- RESULT OF WRONG IMAGES --------------')
for task in tasks:
    print("In {} images:".format(task.split('_')[0]))
    best_acc = 0
    best_method = ''
    for method in methods:
        correct_cnt = 0
        total_cnt = 0
        for user_id in user_dict[exp][method].keys():
            if user_dict[exp][method][user_id]['Quality'] == 'Good':
                for WRONG_BIN_IMAGES in [WRONG_BIN1_IMAGES, WRONG_BIN2_IMAGES, WRONG_BIN3_IMAGES]:
                    correct_cnt += user_dict[exp][method][user_id][task][WRONG_BIN_IMAGES]
        for WRONG_BIN_IMAGES in [WRONG_BIN1_IMAGES, WRONG_BIN2_IMAGES, WRONG_BIN3_IMAGES]:   
            total_cnt += trial_cnt[task][method][WRONG_BIN_IMAGES]
                    
#         print("Correct count: {} on Total count: {}".format(correct_cnt, total_cnt))
        if total_cnt:
            print("Task: {} | Method: {} | Accuracy: {:.2f}%".format(task, method, correct_cnt*100/total_cnt))
            if correct_cnt*100/total_cnt > best_acc:
                best_acc = correct_cnt*100/total_cnt
                best_method = method
    
    print("Answer: The best method is {} with an accuracy of {:.2f}%".format(best_method, best_acc))

In [ ]:
# Q?
print('Q?: Test time vs accuracy?')
time_vs_accuracy = dict()
time_vs_accuracy[exp] = dict()
for method in methods:
    time_vs_accuracy[exp][method] = dict()
    for user_id in user_dict[exp][method].keys():
        correct_cnt = 0
        if user_dict[exp][method][user_id]['Quality'] == 'Good':
            test_time = user_dict[exp][method][user_id]['Test Reaction Time']/60/1000
            for task in tasks:
                correct_cnt += sum(user_dict[exp][method][user_id][task].values())
            time_vs_accuracy[exp][method][test_time] = correct_cnt
            
    time_vs_accuracy[exp][method] = {k: time_vs_accuracy[exp][method][k] for k in sorted(time_vs_accuracy[exp][method])}
    
    print(np.corrcoef(list(time_vs_accuracy[exp][method].keys()), list(time_vs_accuracy[exp][method].values())))
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.scatter(time_vs_accuracy[exp][method].keys(), time_vs_accuracy[exp][method].values())
    plt.title('Task: {} | Method: {}'.format(exp, method))
    plt.ylabel('Correct answers')
    plt.xlabel('Minutes')
    plt.show()
    plt.close()

# print(time_vs_accuracy)
for method in methods:
    print(method)
    print(sum(time_vs_accuracy[exp][method].keys())/len(time_vs_accuracy[exp][method].keys()))
#     print(len(time_vs_accuracy[exp][method].keys()))